In [29]:
import os
import re
import os
import pandas as pd
from astropy.time import Time

In [30]:
# Récupère le chemin du script en cours
base_dir = os.getcwd()

# Construire les chemins relatifs
input_dir = os.path.join(base_dir, "txt")
output_dir = os.path.join(base_dir, "csv")

In [31]:
# Fonction pour traiter un fichier individuel
def process_horizons_file(filepath):
    data = []
    planet_name = None

    with open(filepath, 'r', encoding='utf-8') as file:
        lines = [line.strip() for line in file.readlines()]

    # Trouver le nom de la planète
    for line in lines:
        if "Target body name:" in line:
            planet_name = line.split(":")[1].split('(')[0].strip()
            break

    if not planet_name:
        print(f"⚠️ Impossible de détecter le nom de la planète pour {filepath}")
        return

    # Repérer la section $$SOE à $$EOE
    try:
        start_index = lines.index("$$SOE") + 1
        end_index = lines.index("$$EOE")
    except ValueError:
        print(f"❌ Balises $$SOE ou $$EOE manquantes dans {filepath}")
        return

    # Parcours des lignes contenant les données
    i = start_index
    while i < end_index:
        # Ligne 1 = Julian Date + Date calendrier
        jd_line = lines[i]
        jd = float(jd_line.split()[0])
        time = Time(jd, format='jd', scale='tdb')
        date = time.iso  # 'YYYY-MM-DD HH:MM:SS'
        i += 1

        # Ligne 2 = X, Y, Z
        xyz_line = lines[i]
        xyz_parts = xyz_line.replace('=', '').split()
        x = float(xyz_parts[1])
        y = float(xyz_parts[3])
        z = float(xyz_parts[5])
        i += 1

        vxyz_line = lines[i]

        # Regex qui capture les 3 valeurs de VX, VY, VZ peu importe les espaces
        pattern = r'VX=\s*([-+]?\d*\.\d+E[+-]?\d+)\s*VY=\s*([-+]?\d*\.\d+E[+-]?\d+)\s*VZ=\s*([-+]?\d*\.\d+E[+-]?\d+)'
        match = re.search(pattern, vxyz_line)
        if not match:
            raise ValueError(f"Format inattendu sur la ligne: {vxyz_line}")
        vx = float(match.group(1))
        vy = float(match.group(2))
        vz = float(match.group(3))
        i += 1

        # Ligne 4 = LT, RG, RR (on ignore)
        i += 1

        # Enregistrer dans la liste
        data.append([date, x, y, z, vx, vy, vz])

    # DataFrame final
    df = pd.DataFrame(data, columns=[
        "Date (TDB)", "X (km)", "Y (km)", "Z (km)",
        "VX (km/s)", "VY (km/s)", "VZ (km/s)"
    ])

    # Enregistrement
    os.makedirs(output_dir, exist_ok=True)
    csv_filename = os.path.join(output_dir, f"{planet_name}.csv")
    df.to_csv(csv_filename, index=False)

    print(f"✅ Données de {planet_name} enregistrées dans {csv_filename}")

In [32]:
# Vérification et création du dossier de sortie si inexistant
os.makedirs(output_dir, exist_ok=True)

# Parcourir tous les fichiers .txt dans le dossier input_dir
for file in os.listdir(input_dir):
    if file.endswith(".txt"):
        filepath = os.path.join(input_dir, file)
        process_horizons_file(filepath)

print("🚀 Traitement terminé pour tous les fichiers.")

✅ Données de Sun enregistrées dans /home/ulrich-vianney/Codes/EA-3A-P2/3A-P2-EA-DL/csv/Sun.csv
✅ Données de Neptune enregistrées dans /home/ulrich-vianney/Codes/EA-3A-P2/3A-P2-EA-DL/csv/Neptune.csv
✅ Données de Jupiter enregistrées dans /home/ulrich-vianney/Codes/EA-3A-P2/3A-P2-EA-DL/csv/Jupiter.csv
✅ Données de Mercury enregistrées dans /home/ulrich-vianney/Codes/EA-3A-P2/3A-P2-EA-DL/csv/Mercury.csv
✅ Données de Venus enregistrées dans /home/ulrich-vianney/Codes/EA-3A-P2/3A-P2-EA-DL/csv/Venus.csv
✅ Données de Uranus enregistrées dans /home/ulrich-vianney/Codes/EA-3A-P2/3A-P2-EA-DL/csv/Uranus.csv
✅ Données de Mars enregistrées dans /home/ulrich-vianney/Codes/EA-3A-P2/3A-P2-EA-DL/csv/Mars.csv
✅ Données de Earth enregistrées dans /home/ulrich-vianney/Codes/EA-3A-P2/3A-P2-EA-DL/csv/Earth.csv
✅ Données de Saturn enregistrées dans /home/ulrich-vianney/Codes/EA-3A-P2/3A-P2-EA-DL/csv/Saturn.csv
🚀 Traitement terminé pour tous les fichiers.
